In [1]:
include("../src/Julia.jl")

objc[35292]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[35292]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[35292]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[35292]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/rand_u_1000_1/"

"../../graphs/rand_u_1000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:3
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

for i in 1:3
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.878225 seconds (275.03 k allocations: 11.676 MB)
  0.432456 seconds (123.84 k allocations: 4.983 MB)
  1.856851 seconds (3.17 M allocations: 95.389 MB, 2.72% gc time)
Working on iteration 10 with a-norm error 175.01106427440806
Working on iteration 20 with a-norm error 103.25226258789372
Working on iteration 30 with a-norm error 42.45502379279992
Working on iteration 40 with a-norm error 17.16098325309341
Working on iteration 50 with a-norm error 7.87727154836047
Working on iteration 60 with a-norm error 3.3435640609721218
Working on iteration 70 with a-norm error 1.1221784254154838
Working on iteration 80 with a-norm error 0.46099085119845934
Working on iteration 90 with a-norm error 0.20693641293031545
Working on iteration 100 with a-norm error 0.09225776437863406
Working on iteration 110 with a-norm error 0.03265193407455715
Working on iteration 120 with a-norm error 0.014298278560674971
Working on iteration 130 with a-norm error 0.006765100458266367
Working on iteration 140 wit